# Test scripts

In [ ]:
import sys

print(sys.version)

In [ ]:
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1 0 n1
s space 1 n1 n2
pd pout n2
xaxis laser P lin 1 10 100
"""
kat.parse(code)
out = kat.run()
out.plot()

# Explained Power Sweep Example

![](cavity-powersweep.png)

## Finesse2 basics

* Finesse uses **nodes** to connect components
* `xaxis` is used to modulate (and plot) any value
    * in this case we change the power output of the `laser` component

In [ ]:
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1 0 node0
s space 1 node0 node1
m mirror 0.5 0.5 0 node1 node2
pd photodiode node2

xaxis laser P lin 0 10 10
"""

kat.parse(code)
out = kat.run()
out.plot()

## code deep dive

* `l laser 1 0 node0`
    * `l` laser
    * `laser` name of component
    * `1` P (power in Watts)
    * `0` frequency offset
    * no phase (optional)
    * `node0` connected node
* `s space 1 node0 node1`
    * `s` free space
    * `space` component name
    * `1` length (in meters)
    * optional index of refraction (default 1)
    * `node0` `node1` connected nodes
* `m mirror 0.5 0.5 0 node1 node2`
    * `m` mirror
    * `mirror` component name
    * `0.5` R (reflectivity)
    * `0.5` T (power transmission)
    * `0` phi (tuning in degrees)
    * `node1 node2` connected nodes
* `pd photodiode node2`
    * `pd` photo-diode
    * `photodiode` component name
    * `n2` connected node
* `xaxis l1 P lin 0 10 10`
    * x-axis definition (parameter to tune)
    * `l1` component to tune
    * `P` parameter of component to tune
    * `lin` linear
    * `0` from (Watts)
    * `10` to (Watts)
    * steps

# Simulation of Physical Cavity Setup

![](cavity.png)

* the length of the `cavityspace` is dependent on the focal length of the mirrors
    * in our case the focal length is 10cm so the `cavityspace` will be 20cm
* whether the cavity is in resonance depends on the the exact distance between the two mirrors
    * if it's a multiple of the `laser`'s frequency the cavity will be in resonance
    * in Finesse2 this is represented by the **tuning** (phi) of a mirror

---

Small distance adjustments (nanometers) should be accomplished via changing the tuning of a component, large adjustments via the length parameter of a space.
**Small adjustments will NOT work via the length parameter!**

## High Reflectivity

* `piezomirror` R of 99,7%
* `lambda` for phi is set in [kat.ini](kat.ini) (in our case: `lambda 6.33E-7`, 633 nanometers)
* `piezomirror`: a positive phi moves the mirror `node4` closer to `node3`

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()

reflectivity = 0.997

code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

## Low Reflectivity


* `piezomirror` R of 90%
* a lower reflectivity reduces the power output of the cavity

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *
import matplotlib.pyplot as plt

kat = finesse.kat()
reflectivity = 0.9

code = f"""
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror {reflectivity} {1-reflectivity} 0 node3 node4
pd photodiode node4

xaxis piezomirror phi lin 0 360 10_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)

In [ ]:
from IPython.display import display
import ipywidgets as widgets

switch = True

button = widgets.Button(description="toggle scaling")
output = widgets.Output()

display(button, output)


def on_button_clicked(b):
    global switch
    if switch:
        plt.ylim(top=1259999.864594087)
    else:
        plt.ylim(top=135888.67754189012)
    switch = not switch


button.on_click(on_button_clicked)

## Reflectivity with phi == 0

When **phi** is zero degress the cavity is in perfect resonance.

* phi is fixed to zero
* the reflectivity of `piezomirror` is swept from 90% to 100%
* 1.000 steps
* higher reflectivity leads to exponentially more cavity power output

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis piezomirror R lin .9 1 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)
print(kat.laser.P)

## Laser power at phi==0

* phi is fixed to zero
* the power output of `laser` is swept from 0 to 3 Megawatts
* higher power input leads to a linear increase of cavity power output
* 1.000 steps

In [ ]:
%matplotlib widget

from pykat import finesse
from pykat import finesse
from pykat.commands import *

kat = finesse.kat()
code = """
l laser 1.2M 0 node0
s prespace 0.1 node0 node1
m fixedmirror 0.997 0.003 0 node1 node2
s cavityspace 0.2 node2 node3
m piezomirror .997 .003 0 node3 node4
pd photodiode node4

xaxis laser P lin 0 3M 1_000
"""

kat.parse(code)
out = kat.run()
fig = out.plot(return_fig=True)
fig.set_figheight(5)
fig.set_figwidth(11)
print(kat.laser.P)